# Introdução

Este script tem como objetivo analisar os dados de vibração de um motor, comparando-os com um padrão de referência. A partir dessa comparação, serão identificadas possíveis falhas no funcionamento do motor, como vibrações acima do normal, que podem indicar problemas como eixo torto, folgas nas buchas ou até mesmo falhas mais graves, como o motor fundido.

A análise é realizada em várias etapas:
1. Leitura e visualização dos dados de padrão.
2. Leitura e visualização do dataset completo.
3. Divisão do dataset em pacotes de 256 valores.
4. Classificação de falhas com base na correlação entre os pacotes e o padrão.
5. Interface para o usuário escolher um pacote e visualizar a comparação com o padrão.
6. Identificação de novos padrões de falha a partir dos pacotes com correlação abaixo de 0.80.

# 1. Importação das Bibliotecas Necessárias

Primeiro, importamos as bibliotecas necessárias para a análise de dados e visualização.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. Leitura e Visualização dos Dados de Padrão

Nesta etapa, lemos o arquivo de padrão e plotamos os dados para visualizar o comportamento esperado da vibração do motor.

In [ ]:
# Leitura do arquivo de padrão
caminho_padrao = "PADRAOVIBRACAOMOTORSTREAM009CC.txt"
dados_padrao = np.loadtxt(caminho_padrao)

# Plotagem dos dados de padrão
plt.figure(figsize=(10, 6))
plt.plot(dados_padrao, label="Padrão de Vibração", color="green")
plt.title("Dados de Vibração do Motor - Padrão")
plt.xlabel("Amostras")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()

# 3. Leitura e Visualização do Dataset Completo

Agora, lê-se o dataset completo e plotamos os dados para visualizar o comportamento geral da vibração do motor.

In [ ]:
# Leitura do dataset completo
caminho_dataset = "VIBRACAOMOTORSTREAM009CC.txt"
dados_dataset = np.loadtxt(caminho_dataset)

# Plotagem do dataset completo
plt.figure(figsize=(10, 6))
plt.plot(dados_dataset, label="Dataset de Vibração", color="red")
plt.title("Dados de Vibração do Motor - Dataset Completo")
plt.xlabel("Amostras")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()

# 4. Divisão do Dataset em Pacotes de 256 valores.

O dataset é dividido em pacotes de 256 valores para facilitar a análise e a comparação com o padrão.

In [ ]:
# Divisão do dataset em pacotes de 256 valores
tamanho_pacote = 256
num_pacotes = len(dados_dataset) // tamanho_pacote
pacotes = [dados_dataset[i * tamanho_pacote:(i + 1) * tamanho_pacote] for i in range(num_pacotes)]

print(f"\nO dataset foi dividido em {num_pacotes} pacotes de {tamanho_pacote} valores cada.")


# 5. Classificação de Falhas com base na Correlação

Aqui, definimos uma função para classificar as falhas com base na correlação entre os pacotes e o padrão. Em seguida, calculamos as correlações e classificamos as falhas para cada pacote.

In [ ]:
# Função para classificar falhas com base na correlação
def classificar_falha(correlacao):
    if correlacao > 0.80:
        return "Funcionamento Normal. Motor dentro do padrão de vibração. \nNenhuma falha encontrada."
    elif 0.50 <= correlacao <= 0.80:
        return "Vibração pouco acima do normal. Recomenda-se uma avaliação preventiva. \nPossível eixo torto ou folga nas buchas."
    else:
        return "Vibração muito acima do normal. Recomenda-se uma manutenção imediata. \nPossível falha no cárter ou motor fundido."

# Lista para armazenar pacotes com falha e seus respectivos rótulos
dados_falha = []

# Calculando correlações e atribuindo falhas
for i, pacote in enumerate(pacotes):
    correlacao = np.corrcoef(dados_padrao[:tamanho_pacote], pacote)[0, 1]
    
    # Classificando a falha com base na correlação
    falha = classificar_falha(correlacao)
    
    # Armazenando o pacote e a falha no conjunto de dados
    dados_falha.append({
        "pacote": pacote, 
        "correlacao": correlacao, 
        "falha": falha
    })
    
    # Exibindo a classificação de falha
    print(f"\n\nPACOTE NÚMERO {i+1}: \nCorrelação = {correlacao:.2f} \nFalha: {falha}")

# 6. Interface para o Usuário escolher um pacote

Aqui, o usuário pode escolher um pacote específico para visualizar a comparação com o padrão.

In [ ]:
# Interface para o usuário escolher um pacote
escolha = int(input("Digite o número do pacote que deseja plotar (1 a {}: ".format(num_pacotes)))

# Verificação da escolha do usuário
if 1 <= escolha <= num_pacotes:
    pacote_escolhido = pacotes[escolha - 1]

# Plotagem do padrão e do pacote escolhido no mesmo gráfico
plt.figure(figsize=(10, 6))
plt.plot(dados_padrao[:tamanho_pacote], label="Padrão de Vibração", color="green")
plt.plot(pacote_escolhido, label=f"Pacote {escolha}", color="red", linestyle="dashed")
plt.title(f"Comparação: Padrão x Pacote {escolha}")
plt.xlabel("Amostras")
plt.ylabel("Amplitude")
plt.legend()
plt.grid(True)
plt.show()

# Aplicação de correlação com o Padrão
correlacao = np.corrcoef(dados_padrao[:tamanho_pacote], pacote_escolhido)[0, 1]
print(f"\n\nPACOTE NÚMERO {escolha}: \nCorrelação com o padrão: = {correlacao:.2f} \nFalha: {falha}")

# 7. Identificação de Novos Padrões de Falha

Nesta etapa, identificamos pacotes com correlação abaixo de 0.8 e criamos um novo padrão de falha a partir da média desses pacotes

In [ ]:
# Lista para armazenar pacotes com correlação < 0.8
novos_padroes_falha = []
correlacoes = []
pacotes_falha = []

# Analisando pacotes com correlação < 0.8
for i, pacote in enumerate(pacotes):
    correlacao = np.corrcoef(dados_padrao[:tamanho_pacote], pacote)[0, 1]

    if correlacao < 0.8:
        novos_padroes_falha.append(pacote.tolist()) 
        correlacoes.append(correlacao)
        pacotes_falha.append(i + 1)

# Somente processar se houver pacotes com falha
if novos_padroes_falha:
    novos_padroes_falha = np.array(novos_padroes_falha) 
    limite_diferenca = 0.05

    grupos = []
    grupo_atual = [novos_padroes_falha[0]]
    referencia_correlacao = correlacoes[0]

    for j in range(1, len(novos_padroes_falha)):
        if abs(correlacoes[j] - referencia_correlacao) <= limite_diferenca:
            grupos.append(np.array(grupo_atual))
            grupo_atual = [novos_padroes_falha[j]]
            referencia_correlacao = correlacoes[j]

    if grupo_atual:
        grupos.append(np.array(grupo_atual))

    novos_padroes = [np.mean(grupo, axis=0) for grupo in grupos]
    novo_padrao_falha = np.mean(novos_padroes, axis=0)

    print(f"\nNovo padrão de falha criado com a média dos pacotes com correlação menor que 0.8, sendo eles: \n{', '.join(map(str, pacotes_falha))}.")

    # Plotando o novo padrão de falha
    plt.figure(figsize=(10, 6))
    plt.plot(novo_padrao_falha, label="Novo Padrão de Falha", color="purple")
    plt.title("Novo Padrão de Falha")
    plt.xlabel("Amostras")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Comparando com o padrão original
    plt.figure(figsize=(10, 6))
    plt.plot(dados_padrao[:tamanho_pacote], label="Padrão de Vibração", color="green")
    plt.plot(novo_padrao_falha, label="Novo Padrão de Falha", color="purple", linestyle="dashed")
    plt.title("Comparação: Padrão Original vs Novo Padrão de Falha")
    plt.xlabel("Amostras")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Armazenando o novo padrão de falha (opcional: salvar em um arquivo)
    np.savetxt("novo_padrao_falha.txt", novo_padrao_falha, fmt="%.6f")
    print("\nNovo padrão de falha salvo no arquivo 'novo_padrao_falha.txt'.")
else:
    print("\nNenhum novo padrão de falha foi criado.")